In [1]:
import os
import pandas as pd
import xgboost
import utils
import scoring
from sklearn.model_selection import train_test_split

In [ ]:
DATA_PATH = r"./data/"
train_filename = "train_smaller100.csv.gz"
test_filename = "test_smaller100.csv.gz"

In [ ]:
train,test = utils.load_small_data_csv(DATA_PATH,train_filename,test_filename, utils.SIMPLE_FEATURE_COLUMNS)

# Adding the kink feature
PointResidualstrain,Anglestrain = utils.kink(train)
train['PointResiduals'] = pd.Series(PointResidualstrain, index=train.index)
train['Angles'] = pd.Series(Anglestrain, index=train.index)

PointResidualstest,Anglestest = utils.kink(test)
test['PointResiduals'] = pd.Series(PointResidualstest, index=test.index)
test['Angles'] = pd.Series(Anglestest, index=test.index)


In [ ]:
train_part, validation = train_test_split(train, test_size=0.25, shuffle=True)

In [ ]:
model = xgboost.XGBClassifier(n_jobs=-1)

In [ ]:
model.fit(train_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values,
          train_part.label.values,
          sample_weight=train_part.weight.values)

In [ ]:
validation_predictions = model.predict_proba(validation.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]

In [ ]:
scoring.rejection90(validation.label.values, validation_predictions, sample_weight=validation.weight.values)

In [ ]:
model.fit(train.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values, train.label, sample_weight=train.weight)

In [ ]:
predictions = model.predict_proba(test.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]

In [ ]:
pd.DataFrame(data={"prediction": predictions}, index=test.index).to_csv(
    "sample_submission.csv", index_label="id")